In [ ]:
%matplotlib inline
import os
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import h5py
import json
from collections import defaultdict
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

<h2>Methods</h2>
<br>

In [ ]:
def readJsonData(json_file):
    """
    Reads json_file's data and returns it as a Python dictionary
    Args:
        json_file: path of the json file to be read along with .json extension
    """
    with open(json_file) as data:
        return json.load(data)

In [ ]:
# path to dictionary which maps image ID's to image label classes. Multiple labels is allowed
dict_of_labels = readJsonData("data/image_labels.json")

In [ ]:
def getLabel(file_path):
    ID = os.path.splitext(file_path)[0].split('/')[-1]
    if ID in dict_of_labels:
        return dict_of_labels[ID]
    # The augmented images have id which is the original id followed by '_' followed by other numbers
    if '_' in ID:
        ID = ID[:ID.rfind('_')]
        return dict_of_labels[ID]
    raise AssertionError("ID not in root dictionary")

In [ ]:
common_img_extensions = {'.tif', '.tiff', '.gif', '.jpeg', '.jpg', '.jif', '.jfif',
                         '.jp2', '.jpx', '.j2k', '.j2c', '.fpx', '.pcd', '.png'}

In [ ]:
STANDARD_SIZE=(224,224)

def load_image(filename, verbose=False):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    if verbose==True:
        print( "(%s) changing size from %s to %s" % (filename, str(img.size), str(STANDARD_SIZE)) )
    img = img.resize(STANDARD_SIZE)
    res = np.array(img.getdata(), dtype=np.int16)
    if img.mode != 'RGB':
        if img.mode == 'RGBA':
            res = res[:,0:3]
        else:
            imgc = Image.new('RGB', img.size)
            imgc.paste(img)
            res = np.array(imgc.getdata(), dtype=np.int16)

    return res.reshape(STANDARD_SIZE+(3,))

In [ ]:
def load(rootdir, include):
    """
    Load images from rootdir as numpy arrays.
    
    Args:
        rootdir : path to the directory containing images
        include : the classes you want to include while loading data
    """
    print('loading facets and filenames from {}'.format(rootdir))
    imgs = defaultdict(set)
    
    for root,_,files in os.walk(rootdir):
        for f in files:
            extension = os.path.splitext(f)[1]
            if not extension in common_img_extensions:
                continue
            facet = os.path.split(root)[-1]
            if facet in include:
                ffull = os.path.join(root,f)
                imgs[facet].add(ffull)
    X_res = []
    y_res = []
    for facet, files in imgs.items():
        print( 'processing {}: {} files'.format(facet, len(files)) )
        for f in tqdm(files):
            data = load_image(f)
            label = [facet]
#             if you want multi label settings stored in dictionary loaded above, comment the above line and uncomment
#             the following two lines
#             label = getLabel(f)
#             label = [x for x in label if x in include]
            X_res.append(data)
            y_res.append(label)

    return np.array(X_res),np.array(y_res)

<h2>Load Images</h2>
<br>

In [ ]:
classes = ['cat', 'dog', 'rabbit']

In [ ]:
train_path = "data/train"
validate_path = "data/validate"

In [ ]:
X_train, y_train = load(train_path, classes)

In [ ]:
X_valid, y_valid = load(validate_path, classes)

In [ ]:
train_x = X_train
train_y = MultiLabelBinarizer(classes).fit_transform(y_train)

valid_x = X_valid
valid_y = MultiLabelBinarizer(classes).fit_transform(y_valid)

In [ ]:
print(type(train_x))
print(type(train_y))

In [ ]:
print(train_x.shape)
print(train_y.shape)

In [ ]:
train_h5_name = 'data/train.h5'
valid_h5_name = 'data/validate.h5'

In [ ]:
with h5py.File(train_h5_name, 'w') as hf:
    hf.create_dataset("images",  data=train_x)
    hf.create_dataset("labels", data = train_y)

with h5py.File(valid_h5_name, 'w') as hf:
    hf.create_dataset("images",  data=valid_x)
    hf.create_dataset("labels", data = valid_y)

In [ ]:
with h5py.File(train_h5_name, 'r') as hf:
        X = hf['images'][:]
        y = hf['labels'][:]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True)
X_batch, y_batch = next(datagen.flow(X, y, batch_size=4))
print(X_batch.shape)
print(y_batch)

In [ ]:
# code taken from fastai

def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=8)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
images, labels = X_batch, y_batch
plots(images, titles = labels)